In [44]:
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#import soa data
#trigger event related
tobacoUse_P = pd.DataFrame(pd.read_excel (r'soa-data-cleaned.xlsx', sheet_name='tobaco-P'))
tobacoUse_A = pd.DataFrame(pd.read_excel (r'soa-data-cleaned.xlsx', sheet_name='tobaco-A'))

alcohol_P = pd.DataFrame(pd.read_excel (r'soa-data-cleaned.xlsx', sheet_name='alcohol-P'))
alcohol_A = pd.DataFrame(pd.read_excel (r'soa-data-cleaned.xlsx', sheet_name='alcohol-A'))

weight_P = pd.DataFrame(pd.read_excel (r'soa-data-cleaned.xlsx', sheet_name='weight-P'))
weight_A = pd.DataFrame(pd.read_excel (r'soa-data-cleaned.xlsx', sheet_name='weight-A'))

bloodpressure_P = pd.DataFrame(pd.read_excel (r'soa-data-cleaned.xlsx', sheet_name='bloodpressure-P'))
bloodpressure_A = pd.DataFrame(pd.read_excel (r'soa-data-cleaned.xlsx', sheet_name='bloodpressure-A'))

diabete_P = pd.DataFrame(pd.read_excel (r'soa-data-cleaned.xlsx', sheet_name='diabete-P'))
diabete_A = pd.DataFrame(pd.read_excel (r'soa-data-cleaned.xlsx', sheet_name='diabete-A'))

cholesterol_P = pd.DataFrame(pd.read_excel (r'soa-data-cleaned.xlsx', sheet_name='cholesterol-P'))
cholesterol_A = pd.DataFrame(pd.read_excel (r'soa-data-cleaned.xlsx', sheet_name='cholesterol-A'))

phyAct_P = pd.DataFrame(pd.read_excel (r'soa-data-cleaned.xlsx', sheet_name='phyAct-P'))
phyAct_A = pd.DataFrame(pd.read_excel (r'soa-data-cleaned.xlsx', sheet_name='phyAct-A'))

misAct_P = pd.DataFrame(pd.read_excel (r'soa-data-cleaned.xlsx', sheet_name='misAct-P'))
misAct_A = pd.DataFrame(pd.read_excel (r'soa-data-cleaned.xlsx', sheet_name='misAct-A'))

#import population data
pop_P = pd.DataFrame(pd.read_excel (r'soa-data-cleaned.xlsx', sheet_name='pop-P'))
pop_A = pd.DataFrame(pd.read_excel (r'soa-data-cleaned.xlsx', sheet_name='pop-A'))
print(np.round(np.array(pop_P['age percent(15-75+)'])[0:8], 2))

[0.07 0.07 0.16 0.14 0.12 0.1  0.05 0.02]


In [3]:
#sampling for population with respect to age, gender
def generate_sample_pop(city, size = 10000, m_rate = 0.5, f_rate = 0.5):
    assert(size > 100), "population size too small!"
    
    pop_dis = pd.DataFrame(pd.read_excel (r'soa-data-cleaned.xlsx', sheet_name='pop-' + city))
    pop_dis = np.round(np.array(pop_dis['age percent(15-75+)'])[0:8],2)
    
    pop_rep_m = pop_dis * m_rate * size
    pop_rep_m = pop_rep_m.astype(int)
    pop_rep_f = pop_dis * f_rate * size
    pop_rep_f = pop_rep_f.astype(int)
    age_band = np.array(["15-19", "20-24", "25-34", "35-44", "45-54", "55-64", "65-74", "75-99"])
    sampling_ages_M = np.repeat(age_band, pop_rep_m, axis=0)
    sampling_ages_F = np.repeat(age_band, pop_rep_f, axis=0)
    df = pd.DataFrame({'male' :list(sampling_ages_M), 'female':list(sampling_ages_M)}, columns = ["male", "female"])
    return df

    
millionP = generate_sample_pop('P', 1000000)

In [4]:
#select a subset of poplulation from the random generated population

#envir var for M/FM's age distri
female_sample = pd.DataFrame()
male_sample = pd.DataFrame()
def bootstrap_pop(df, size):
    #random generate a rate between F and M wrt to the true rate
    #need randomness as gender is divided up
    #change 0.5 if needr
    rand = np.random.binomial(size, 0.5, 1)
    m_rate = round(np.average(rand),0)
    f_rate = size - m_rate
    print(m_rate)
    male_dis = np.array(df['male'])
    sample_dis_m = random.choices(list(male_dis), k=int(m_rate))
    male_sample['age'] = sample_dis_m
    male_sample["count"] = [1] * int(m_rate)
    
    female_dis = np.array(df['female'])
    sample_dis_fm = random.choices(list(female_dis), k=int(f_rate))
    female_sample['age'] = sample_dis_fm
    female_sample["count"] = [1] * int(f_rate)


bootstrap_pop(millionP, 10000)
sp = male_sample.groupby(['age']).count()

# ind = str(list(sp.index)[0])

int(sp.loc['15-19'])


4986.0


484

In [9]:
#create sub DF for each of the age range
#tabaco use were divided into three zones: non-smoker, daily smoker and occasional smoker
#each of which will follow the distribution as suggested within specific age range

#envir var store 8 DF for each age range
dic_df = {}

def sample_taboco(city = "P"):
    age_band = male_sample.groupby(['age']).count().index.tolist()
    # number of pp in the sample wrt different age band
    gb_m = male_sample.groupby(['age']).count()
    gb_fm = female_sample.groupby(['age']).count()
    
    for i in np.arange(len(age_band)):
        if city == "P"
            distri = tobacoUse_P.iloc[[i]].values.tolist()[0][1:4]
        else:
            distri = tobacoUse_A.iloc[[i]].values.tolist()[0][1:4]
        df = pd.DataFrame()
        
        for j in np.arange(3):
        # j = 0, non-smoker at ageband[i]
        # j = 1, daily smoker at ageband[i]
        # j = 2, occ smoker at ageband[i]

            num = int(round(distri[j] * int(gb_m.loc[age_band[i]]), 0))
            ls = [1] * num + [0] * int(gb_m.loc[age_band[i]] - num)
#             print(len(ls))
            
            rd_ls = random.sample(ls, len(ls))
            if j == 0:
                df['non-smoker'] = rd_ls
            elif j == 1:
                df['daily-smoker'] = rd_ls
            else:
                df['occ-smoker'] = rd_ls
        
        dic_df[age_band[i]] = df
#             print(sample15To75[i].head(2))
#             print(sample15To75[i+5].head(2))
              
    return dic_df

ls = sample_taboco()

In [19]:
def sample_alcho(city = "P"):
    # a list that ['x-x', 'y-y',...]
    age_band = male_sample.groupby(['age']).count().index.tolist()
    # number of pp in the sample wrt different age band
    gb_m = male_sample.groupby(['age']).count()
    gb_fm = female_sample.groupby(['age']).count()
    
    for i in np.arange(len(age_band)):
        if city == "P":
            distri = alcohol_P.iloc[[i]].values.tolist()[0][1:7]
        else:
            distri = alcohol_A.iloc[[i]].values.tolist()[0][1:7]
        
        for j in np.arange(6):
        # j = 0, everyday at ageband[i]
        # j = 1, every week at ageband[i]
        # j = 2, every month at ageband[i]
        # j = 3, < 1 per month at ageband[i]
        # j = 4, < 1 last year at ageband[i]
        # j = 5, never at ageband[i]
            df = dic_df[age_band[i]]
            num = int(round(distri[j] * int(gb_m.loc[age_band[i]]), 0))
            ls = [1] * num + [0] * int(gb_m.loc[age_band[i]] - num)
#             print(len(ls))
            
            rd_ls = random.sample(ls, len(ls))
            if j == 0:
                df['drink every day'] = rd_ls
            elif j == 1:
                df['drink every week'] = rd_ls
            elif j == 2:
                df['drink every month'] = rd_ls
            elif j == 3:
                df['drink <1 per month'] = rd_ls
            elif j == 4:
                df['drink <1 last year'] = rd_ls
            else:
                df['never drink'] = rd_ls
        
                
#             print(sample15To75[i].head(2))
#             print(sample15To75[i+5].head(2))
sample_alcho()

In [14]:
alcohol_P.iloc[[0]].values.tolist()[0][1:7]

[0.0025, 0.0335, 0.1053, 0.1035, 0.1, 0.6552]

In [11]:
print(alcohol_P)

  Age Band  Every Day  Every Week  Every Month  Less Than Once a Month  \
0    15–19     0.0025      0.0335       0.1053                  0.1035   
1    20–24     0.0145      0.1225       0.2575                  0.1493   
2    25–34     0.0313      0.1615       0.2725                  0.1483   
3    35–44     0.0380      0.1720       0.2608                  0.1435   
4    45–54     0.0543      0.1748       0.2313                  0.1500   
5    55–64     0.0533      0.1343       0.2048                  0.1460   
6    65–74     0.0393      0.0958       0.1588                  0.1388   
7      75+     0.0243      0.0520       0.1095                  0.1050   

   Not in the Last 12 Months   Never  Never or Not in the Last 12 Months  
0                     0.1000  0.6552                              0.7552  
1                     0.0988  0.3574                              0.4562  
2                     0.1113  0.2751                              0.3864  
3                     0.1150  0.2

In [161]:
dd=pd.DataFrame([[1, 2], [4, 5], [7, 8]],
      index=['cobra', 'viper', 'sidewinder'],
      columns=['max_speed', 'shield'])
dd.index

Index(['cobra', 'viper', 'sidewinder'], dtype='object')

In [119]:
#COMPUTE Y=X*r+Y*SQRT(1-r**2)
r = 0.9
x = np.array([0] * 35 + [1] * 65) #alcho
y = np.array([0] * 70 + [1] * 30) #smoke

cov = np.cov(np.array([[0] * 35 + [1] * 65,[0] * 70 + [1] * 30]))
# y2 = x*r + y**(1 - r**2)
# y2 
cov_ab = 0.75 * (0.22**0.5)*(0.21**0.5)
cov[0][1] = cov_ab
cov[1][0] = cov_ab
y = np.random.multivariate_normal(np.array([0.65, 0.3]), cov, size=100)
# plt.plot(y)
# y2 = y.T
# for x in y2[0]:
#     if x > 0:
#         x = 1
#     else:
#         x = 0
# y2
y.T

count = 0
for x in np.arange(100):
    if y.T[0][x] > 0.75 and y.T[1][x] > 0.75:
        count += 1

count

21